In [1]:
import os
import sys

from rich import print

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"✅ Project root added to path: {project_root}")

✅ Project root added to path: /Users/guchen/repo_own/langgraph_project_template/tests/app/common

In [2]:
from typing import Optional
from langchain_core.documents import Document
from langchain_core.documents.compressor import BaseDocumentCompressor
from langchain_community.vectorstores import FAISS
from src.app.common.models import init_embedding_model


def test_embedding_retrieve(
    query: str,
    reranker: Optional[BaseDocumentCompressor] = None
):
    """
    一个灵活的 RAG 检索函数，使用新的文档和查询来凸显 Reranker 的效果。
    """
    # ------------------ 新的文档和查询 ------------------
    docs = [
        Document(
            page_content="尼古拉·特斯拉是一位伟大的发明家，他在19世纪末对交流电（AC）系统的设计和推广做出了关键贡献，这为现代全球电力网络奠定了基础。",
            metadata={"source": "history_pedia"}
        ),
        Document(
            page_content="苹果公司（Apple Inc.）宣布，计划在未来五年内，投资超过4300亿美元，用于在美国发展下一代芯片技术和建设新的数据中心。这项投资计划将专注于半导体研发和供应链强化。",
            metadata={"source": "tech_news_article_1"}
        ),
        Document(
            page_content="特斯拉公司（Tesla, Inc.）公布了最新的季度财报，显示其电动汽车销量再创新高。CEO埃隆·马斯克对公司的电力技术和未来发展充满信心。",
            metadata={"source": "finance_news"}
        ),
        Document(
            page_content="特斯拉 Model S Plaid 车型采用三电机配置，其核心是先进的交流感应电动机技术，能提供惊人的加速性能和电力效率。",
            metadata={"source": "car_manual"}
        ),
        Document(
            page_content="营养学家发布了一份关于水果“苹果”的投资健康计划。该计划建议，每日投资一个苹果可以有效降低心血管疾病风险。许多家庭主妇已经开始了这个健康的投资计划。",
            metadata={"source": "health_blog_post_2"}
        ),
        Document(
            page_content="要缓解大型语言模型的幻觉问题，可以采用多种策略。一种常见的方法是引入知识图谱进行事实校验。此外，使用RAG（检索增强生成）架构，让模型参考外部可信数据源，也能显著减少不准确信息的生成。",
            metadata={"source": "research_paper_summary_3"}
        ),
        Document(
            page_content="大型语言模型（LLM）的幻觉问题是一个严峻的挑战。模型可能会凭空捏造事实、引用不存在的来源，这在金融、医疗等关键领域的应用中构成了巨大风险。幻觉的根本原因在于模型训练数据中的噪声和其生成机制的概率性。",
            metadata={"source": "llm_introduction_4"}
        ),
        Document(
            page_content="Pandas是Python中一个功能强大的数据分析库。它提供了DataFrame和Series两种核心数据结构，非常适合处理表格数据。使用Pandas，你可以轻松地进行数据清洗、转换、聚合和可视化。",
            metadata={"source": "python_data_science_intro_6"}
        ),
        Document(
            page_content="在Pandas中，你可以使用 .dropna() 方法来删除包含缺失值（NaN）的行或列。例如，要删除任何含有NaN的行，只需在你的DataFrame上调用 df.dropna(axis=0, how='any') 即可。",
            metadata={"source": "pandas_cookbook_5"}
        ),
        Document(
            page_content="托马斯·爱迪生是直流电（DC）的主要倡导者，他与支持交流电的尼古拉·特斯拉之间曾有过著名的“电流战争”。",
            metadata={"source": "history_pedia"}
        ),
        Document(
            page_content="为您的电动汽车充电很简单。只需将充电枪连接到车辆的充电端口，电力就会开始从电网流入电池。",
            metadata={"source": "ev_charging_guide"}
        ),
    ]
    # ---------------------------------------------------

    # 使用您指定的 Ollama 嵌入模型
    embeddings = init_embedding_model(model="qwen3-embedding:8b", model_provider="ollama")
    print(f"{embeddings.__class__.__mro__}")

    vector_store = FAISS.from_documents(docs, embeddings)
    print(f"{vector_store.__class__.__mro__}")

    # 基础检索器，我们让它返回更多的结果（k=4），给 reranker 留出排序空间
    retriever = vector_store.as_retriever(search_kwargs={"k": 4})

    print(f"--- 正在执行查询: '{query}' ---")
    if reranker:
        print(f"--- 模式: 已启用 {reranker.__class__.__name__} ---")
        print(f"-- {reranker.__class__.__mro__} ---")
    else:
        print("--- 模式: 未使用 Reranker (基础检索) ---")

    retrieved_docs = retriever.invoke(query)

    # 后处理 Rerank 步骤
    if reranker is not None:
        # reranker.compress_documents 需要一个文档列表和查询字符串
        retrieved_docs = reranker.compress_documents(retrieved_docs, query)

    print("检索到的结果:")
    for i, doc in enumerate(retrieved_docs):
        print(f"  文档 {i+1}: {doc.page_content} (来源: {doc.metadata['source']})")


## `src.common.models.rerank_models`

In [6]:
from src.app.common.models import init_rerank_model

query = "如何处理大模型的幻觉问题"

#reranker = init_rerank_model(model="gte-rerank-v2", model_provider="dashscope")
#reranker = init_rerank_model(model="qwen3-reranker-0.6b", model_provider="xinference")
#reranker = init_rerank_model(model="BAAI/bge-reranker-v2-m3", model_provider="siliconflow")
reranker = init_rerank_model(model="qwen3-reranker-8b", model_provider="dmxapi")

test_embedding_retrieve(query=query)
test_embedding_retrieve(query=query, reranker=reranker)

(<class 'langchain_ollama.embeddings.OllamaEmbeddings'>, <class 'pydantic.main.BaseModel'>, <class 
'langchain_core.embeddings.embeddings.Embeddings'>, <class 'abc.ABC'>, <class 'object'>)

(<class 'langchain_community.vectorstores.faiss.FAISS'>, <class 'langchain_core.vectorstores.base.VectorStore'>, 
<class 'abc.ABC'>, <class 'object'>)

--- 正在执行查询: '如何处理大模型的幻觉问题' ---

--- 模式: 未使用 Reranker (基础检索) ---

检索到的结果:

...

-- (<class 'src.app.common.models.dmxapi.rerank.DMXAPIRerank'>, <class 
'langchain_core.documents.compressor.BaseDocumentCompressor'>, <class 'pydantic.main.BaseModel'>, <class 
'abc.ABC'>, <class 'object'>) ---

检索到的结果:

文档 1: 
要缓解大型语言模型的幻觉问题，可以采用多种策略。一种常见的方法是引入知识图谱进行事实校验。此外，使用RAG（检索增强生
成）架构，让模型参考外部可信数据源，也能显著减少不准确信息的生成。 (来源: research_paper_summary_3)

文档 2: 
营养学家发布了一份关于水果“苹果”的投资健康计划。该计划建议，每日投资一个苹果可以有效降低心血管疾病风险。许多家庭主
妇已经开始了这个健康的投资计划。 (来源: health_blog_post_2)

文档 3: 特斯拉 Model S Plaid 
车型采用三电机配置，其核心是先进的交流感应电动机技术，能提供惊人的加速性能和电力效率。 (来源: car_manual)

In [ ]:
# model_provider = dashscope_rerank
from langchain_community.document_compressors import DashScopeRerank
dashscope_rerank = DashScopeRerank(model="gte-rerank-v2")

test_embedding_retrieve(query=query)
print("\n" + "="*50 + "\n")
test_embedding_retrieve(query=query, reranker=dashscope_rerank)

In [ ]:
# xinference rerank
from langchain_xinference import XinferenceRerank

xinference_rerank = XinferenceRerank(model_uid="bge-reranker-v2-m3", server_url=os.getenv("XINFERENCE_SERVER_URL"))
test_embedding_retrieve(query=query)
test_embedding_retrieve(query=query,reranker=xinference_rerank)


In [ ]:
# siliconflow rerank
from src.app.common.models.siliconflow.rerank import SiliconflowRerank

siliconflow_rerank = SiliconflowRerank(model="BAAI/bge-reranker-v2-m3", instruction="检索有关特斯拉汽车的文档")

test_embedding_retrieve(query=query)
test_embedding_retrieve(query=query,reranker=siliconflow_rerank)

In [ ]:
# dmxapi rerank